# WikiStory 


### Setup

Libraries and define constants.

In [4]:
import pandas as pd
import concurrent.futures
import json
import numpy as np
import shutil
import pathlib

### Convert to parquet

First a converesion from csv is parquet makes everything faster. If the file is in place, you don't need to run it again.

In [ ]:
csv_data = pd.read_csv('data.csv', quotechar ='|')
csv_data.to_parquet('data/input.parquet', engine='pyarrow')

### Add days to parquet
We need to add days to our parquet as that's how we will do clustering of words for later rendering

In [ ]:
input_data = pd.read_parquet('data/input.parquet', engine='pyarrow')

input_data['timestamp'] = pd.to_datetime(input_data['timestamp'])
input_data['day'] = input_data.timestamp.dt.date

In [ ]:
print(input_data.head())

In [ ]:
input_data.to_parquet('data/with_date.parquet',engine='pyarrow')

### Cleanup and Transformation

In [3]:
with_dates = pd.read_parquet('data/with_date.parquet')
# with_dates = with_dates[with_dates.page_title != 'Tartışma:Anasayfa']
grouped_by_date_and_title = with_dates.groupby(['day','page_title']).count()
grouped_by_date_and_title = grouped_by_date_and_title.reset_index()

### Filter out unnecessary words

In [4]:
words_to_filter=[
    "Vikipedi",
    "Kullanıcı:",
    "wiki:",
    "mesaj:",
    "Anasayfa",
    "Tartışma:",
    "Kategori:",
    "Şablon:"
]

grouped_by_date_and_title["ignore"] = False

for word in words_to_filter:
    grouped_by_date_and_title["ignore"] = grouped_by_date_and_title["ignore"] | grouped_by_date_and_title.page_title.str.contains(word)




grouped_by_date_and_title = grouped_by_date_and_title.loc[grouped_by_date_and_title.ignore == False]

### Save Results as Parquet

In [5]:
grouped_by_date_and_title.to_parquet('data/grouped.parquet', engine='pyarrow')

### Save daily JSON's

In [6]:
grouped_by_date_and_title = pd.read_parquet('data/grouped.parquet')

In [7]:
more_than_two = grouped_by_date_and_title[grouped_by_date_and_title['page_id']>1]
more_than_two = more_than_two.reset_index()
grouped_by_day = more_than_two.groupby('day')
grouped_by_day.head()

,index,day,page_title,page_id,page_ns,revision_id,timestamp,contributor_id,contributor_name,bytes,ignore
0,27,2003-08-02,Karl Marx,4,4,4,4,4,4,4,False
1,28,2003-08-02,Linux,5,5,5,5,5,5,5,False
2,30,2003-08-02,Mustafa Suphi,3,3,3,3,3,3,3,False
3,33,2003-08-03,Karl Marx,2,2,2,2,2,2,2,False
4,34,2003-08-04,Beşiktaş JK,3,3,3,3,3,3,3,False
...,...,...,...,...,...,...,...,...,...,...,...
2067101,13306721,2020-03-01,1 Mart,2,2,2,2,2,2,2,False
2067102,13306723,2020-03-01,1367,2,2,2,2,2,2,2,False
2067103,13306724,2020-03-01,14. İnsansız Uçak Sistemleri Üs Meydan Komutan...,3,3,3,3,3,3,3,False
2067104,13306727,2020-03-01,1818 Antlaşması,2,2,2,2,2,2,2,False


##### Cleanup data folder

In [2]:
amcharts_data_path = "../../nodejs/wikistoryweb/public/data"
shutil.rmtree(amcharts_data_path, ignore_errors=True)
pathlib.Path(amcharts_data_path).mkdir(parents=True, exist_ok=True)


NameError: name 'shutil' is not defined

In [3]:
def write_head(arg):
    index = arg[0]
    group = arg[1]
    filename = index
    # tags = group.apply(lambda x: {x.page_id})
    df = group.rename(columns={"page_id": "revisions", "page_title":"title"})
    df = df[["title", "revisions"]]
    mean = np.mean(df["revisions"])
    df = df[df["revisions"] >= mean]
    df[:100].to_json(f'{amcharts_data_path}/{index}.json', orient='records')


    
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
   executor.map(write_head, grouped_by_day)
    
dates = list(map(lambda x: x[0].strftime("%Y-%m-%d"), grouped_by_day.day))
    
index_dict = {'dates': dates}
with open(f'{amcharts_data_path}/index.json',"w+") as outfile:
  json.dump(index_dict, outfile)


NameError: name 'concurrent' is not defined